In [3]:
import numpy as np
import pywt
import os
from PIL import Image
from scipy.fftpack import dct
from scipy.fftpack import idct
from math import log10, sqrt

#current_path = str(os.path.dirname(__file__))  

image = 'images/chihuahua.webp'   
watermark = 'images/watermark.jpg' 

def convert_image(image_name, size):
    img = Image.open(image_name).resize((size, size), 1)
    img = img.convert('L')
    img.save(image_name)

    # image_array = np.array(img)
    # image_array = np.float32(image_array) 
    # image_array /= 255 
    image_array = np.array(img.getdata(), dtype=np.float64).reshape((size, size))
    #print(image_array[0][0])               #qrcode white color = 1.0
    #print(image_array[10][10])             #qrcode black color = 0.0  

    return image_array

def process_coefficients(imArray, model, level):
    #pywt - wavelet transforms in Python: Multilevel 2D Discrete Wavelet Transform.
    coeffs=pywt.wavedec2(data = imArray, wavelet = model, level = level)
    print('Coeffs: ', coeffs[0].__len__())    
    
    coeffs_H=list(coeffs)
    
    return coeffs_H
            
    
def embed_watermark(watermark_array, orig_image):
    watermark_array_size = watermark_array[0].__len__()
    
    #ravel(): return a contiguous flattened 1D array
    watermark_flat = watermark_array.ravel()
    
    ind = 0

    for x in range (0, orig_image.__len__(), 8):
        for y in range (0, orig_image.__len__(), 8):
            if ind < watermark_flat.__len__():
                subdct = orig_image[x:x+8, y:y+8]
                subdct[5][5] = watermark_flat[ind]
                orig_image[x:x+8, y:y+8] = subdct
                ind += 1 
    return orig_image
      


def apply_dct(image_array):
    size = image_array[0].__len__()
    all_subdct = np.empty((size, size))
    for i in range (0, size, 8):
        for j in range (0, size, 8):
            subpixels = image_array[i:i+8, j:j+8]
            subdct = dct(dct(subpixels.T, norm="ortho").T, norm="ortho")
            all_subdct[i:i+8, j:j+8] = subdct

    return all_subdct


def inverse_dct(all_subdct):
    size = all_subdct[0].__len__()
    all_subidct = np.empty((size, size))
    for i in range (0, size, 8):
        for j in range (0, size, 8):
            subidct = idct(idct(all_subdct[i:i+8, j:j+8].T, norm="ortho").T, norm="ortho")
            all_subidct[i:i+8, j:j+8] = subidct

    return all_subidct


def get_watermark(dct_watermarked_coeff, watermark_size):
    # watermark = [[0 for x in range(watermark_size)] for y in range(watermark_size)] 

    subwatermarks = []

    for x in range (0, dct_watermarked_coeff.__len__(), 8):
        for y in range (0, dct_watermarked_coeff.__len__(), 8):
            coeff_slice = dct_watermarked_coeff[x:x+8, y:y+8]
            subwatermarks.append(coeff_slice[5][5])

    watermark = np.array(subwatermarks).reshape(watermark_size, watermark_size)

    return watermark


def recover_watermark(image_array, model='haar', level = 1):
    coeffs_watermarked_image = process_coefficients(image_array, model, level=level)
   
    dct_watermarked_coeff = apply_dct(coeffs_watermarked_image[0])
    
    watermark_array = get_watermark(dct_watermarked_coeff, 128)

    # watermark_array *= 255;
    watermark_array =  np.uint8(watermark_array)

    #Save result
    img = Image.fromarray(watermark_array)
    img.save('./result/recovered_watermark.jpg')


def print_image_from_array(image_array, name):
    # image_array *= 255;
    # image_array =  np.uint8(image_array)
    image_array_copy = image_array.clip(0, 255)
    image_array_copy = image_array_copy.astype("uint8")
    img = Image.fromarray(image_array_copy)
    img.save('./result/' + name)
    
def PSNR(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

def w2d(img):
    model = 'haar'
    level = 1
    image_array = convert_image(image, 2048)
    watermark_array = convert_image(watermark, 128)

    coeffs_image = process_coefficients(image_array, model, level=level)
    #print('Length of coeffes',len(coeffs_image))
    
    print_image_from_array(coeffs_image[0], 'LL_after_DWT.jpg')

    dct_array = apply_dct(coeffs_image[0])
    print_image_from_array(dct_array, 'LL_after_DCT.jpg')

    dct_array = embed_watermark(watermark_array, dct_array)
    print_image_from_array(dct_array, 'LL_after_embeding.jpg')

    coeffs_image[0] = inverse_dct(dct_array)
    print_image_from_array(coeffs_image[0], 'LL_after_IDCT.jpg')


    # reconstruction
    #waverec2 = Multilevel 2D Inverse Discrete Wavelet Transform.
    image_array_H=pywt.waverec2(coeffs_image, model)
    print_image_from_array(image_array_H, 'image_with_watermark.jpg')

    # recover images
    recover_watermark(image_array = image_array_H, model=model, level = level)
    
    #Calculate psnr
    #original = Image.open('mis1.jpg')
    original = image_array
    watermarked = image_array_H
    value = PSNR(original, watermarked)
    print('PSNR value is',value, 'dB')

In [5]:
w2d("test")

Coeffs:  1024
Coeffs:  1024
PSNR value is 26.18510877211094 dB
